<a href="https://colab.research.google.com/github/Lukehsu1999/GYAFC-Formality-Transfer-and-Classification/blob/main/GYAFC_Bert2Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets==1.0.2

import datasets
import transformers

     |████████████████████████████████| 1.4MB 13.7MB/s 
     |████████████████████████████████| 2.9MB 47.7MB/s 
     |████████████████████████████████| 890kB 51.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ec3feddb47c4fae329d5e711ee34ccc587b3209a3f585525597fbe8da48c0d62
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.8MB 12.2MB/s 
     |████████████████████████████████| 17.7MB 201kB/s 
     |████████████████████████████████| 245kB 52.1MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

Get dataset

put your path below

In [ ]:
path_to_formal_train_set="/content/drive/MyDrive/CityU/FYP/Datasets/GYAFC/GYAFC_Corpus/Entertainment_Music/train/formal"
path_to_informal_train_set="/content/drive/MyDrive/CityU/FYP/Datasets/GYAFC/GYAFC_Corpus/Entertainment_Music/train/informal"

In [ ]:
#put your path
train_formal_corpus=open(path_to_formal_train_set,"r")
formal_lines=train_formal_corpus.readlines()
train_informal_corpus=open(path_to_informal_train_set,"r")
informal_lines=train_informal_corpus.readlines()

In [ ]:
from datasets import Dataset

train_data_map={"id":[],"formal":[],"informal":[]}
for i in range(0,20000):
  train_data_map["id"].append(i)
  train_data_map["formal"].append(formal_lines[i])
  train_data_map["informal"].append(informal_lines[i])
  
train_data=Dataset.from_dict(train_data_map)

val_data_map={"id":[],"formal":[],"informal":[]}
for i in range(20000,22000):
  val_data_map["id"].append(i)
  val_data_map["formal"].append(formal_lines[i])
  val_data_map["informal"].append(informal_lines[i])

val_data=Dataset.from_dict(val_data_map)

Process data to model inputs

In [ ]:
batch_size=4  # 4, the max size colab can handle
encoder_max_length=512
decoder_max_length=512

In [ ]:
def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["informal"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["formal"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [ ]:

train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["id", "formal", "informal"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)
print(train_data)


val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["id", "formal", "informal"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


Dataset(features: {'attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'decoder_attention_mask': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'decoder_input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'input_ids': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'labels': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}, num_rows: 20000)


Warm-starting the Encoder-Decoder Model

In [ ]:
from transformers import EncoderDecoderModel
#if you want to start from a checkpoint: 
bert2bert=EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/CityU/FYP/Bert2Bert_Checkpoints/20K_Ep2")
#if you want to build one from start:
#bert2bert = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased", "bert-base-uncased")

In [ ]:
# set special tokens
bert2bert.config.decoder_start_token_id = tokenizer.bos_token_id
bert2bert.config.eos_token_id = tokenizer.eos_token_id
bert2bert.config.pad_token_id = tokenizer.pad_token_id

# sensible parameters for beam search
bert2bert.config.vocab_size = bert2bert.config.decoder.vocab_size
bert2bert.config.max_length = 142
bert2bert.config.min_length = 2 
bert2bert.config.no_repeat_ngram_size = 3
bert2bert.config.early_stopping = True
bert2bert.config.length_penalty = 2.0
bert2bert.config.num_beams = 4

Fine-Tuning Warm-Started Encoder-Decoder Models

In [ ]:
%%capture
!rm seq2seq_trainer.py
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/seq2seq/seq2seq_trainer.py

!pip install git-python==1.0.3
!pip install sacrebleu==1.4.12
!pip install rouge_score

In [ ]:
from seq2seq_trainer import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
@dataclass
class Seq2SeqTrainingArguments(TrainingArguments):
    label_smoothing: Optional[float] = field(
        default=0.0, metadata={"help": "The label smoothing epsilon to apply (if not zero)."}
    )
    sortish_sampler: bool = field(default=False, metadata={"help": "Whether to SortishSamler or not."})
    predict_with_generate: bool = field(
        default=False, metadata={"help": "Whether to use generate to calculate generative metrics (ROUGE, BLEU)."}
    )
    adafactor: bool = field(default=False, metadata={"help": "whether to use adafactor"})
    encoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Encoder layer dropout probability. Goes into model.config."}
    )
    decoder_layerdrop: Optional[float] = field(
        default=None, metadata={"help": "Decoder layer dropout probability. Goes into model.config."}
    )
    dropout: Optional[float] = field(default=None, metadata={"help": "Dropout probability. Goes into model.config."})
    attention_dropout: Optional[float] = field(
        default=None, metadata={"help": "Attention dropout probability. Goes into model.config."}
    )
    lr_scheduler: Optional[str] = field(
        default="linear", metadata={"help": f"Which lr scheduler to use."}
    )

In [ ]:
# load rouge for validation
rouge = datasets.load_metric("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

Training <br>
note that downloading checkpoint from colab to local takes roughly 40 mins

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./",#maybe ./ is better, cause directly linking to cloud will cause trash overflow
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
    #evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=1000,  #2, set to 1000 for full training
    save_steps=500,  #16 set to 500 for full training
    eval_steps=800,  #4 set to 8000 for full training
    warmup_steps=1,  #1 set to 2000 for full training
    #max_steps=16, # delete for full training
    overwrite_output_dir=True,
    save_total_limit=3,
    fp16=True, 
    num_train_epochs=1
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=bert2bert,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

AttributeError: ignored

<h1>Tryout</h1>

In [1]:
!pip install transformers
!pip install datasets==1.0.2

import datasets
import transformers
import torch

     |████████████████████████████████| 1.5MB 10.0MB/s 
     |████████████████████████████████| 890kB 31.1MB/s 
     |████████████████████████████████| 2.9MB 40.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=24dcd6b0afec03d97a89c9b92c48b1b5cfc4cc332d80de2bc16bf8415822eb9c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.8MB 10.8MB/s 
     |████████████████████████████████| 17.7MB 152kB/s 
     |████████████████████████████████| 245kB 39.5MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [2]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

In [3]:
path_to_demo_model=""

In [4]:
from transformers import EncoderDecoderModel
#if you want to start from a checkpoint: 
demo_model=EncoderDecoderModel.from_pretrained(path_to_demo_model)

In [5]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
demo_model.to(device)

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [7]:
demo_sentence='Lets watcha movie together lol'
demo_input=tokenizer(demo_sentence, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
demo_ids=demo_input.input_ids.to("cuda")
demo_attention_mask=demo_input.attention_mask.to("cuda")

outputs=demo_model.generate(demo_ids)
output_str=tokenizer.decode(outputs[0],skip_special_tokens=True)

print(output_str)

we should watch a movie together.


Not so sure whether this is a good movie. -> <br>
Lets watcha movie together lol->

In [ ]:
model.save()